### 하와이 메타데이터 불러오기

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
!mkdir unzip_path
!cd unzip_path # 압축을 풀 폴더 경로
!unzip -qq '/content/drive/MyDrive/ASAC/Project_GoogleMap/meta-Hawaii.json.gz' # 압축파일 경로

In [0]:
import gzip

# JSON 데이터를 파싱하는 함수
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

# 데이터 경로
data_path = '/content/drive/MyDrive/ASAC/Project_GoogleMap/meta-Hawaii.json.gz'

# 데이터를 리스트에 저장
data = []
for item in parse(data_path):
    data.append(item)

# 데이터를 판다스 데이터프레임으로 변환
df = pd.DataFrame(data)

In [0]:
df.head()

## gmap_id 수와 unique한 gmap_id 수가 달라 확인 필요

In [0]:
df.info()

In [0]:
df['gmap_id'].nunique()

In [0]:
# 1. 전체 행의 수
total_rows = df.shape[0]

# 2. 고유한 gmap_id의 개수
unique_gmap_ids = df['gmap_id'].nunique()

# 3. 중복된 gmap_id 찾기
duplicated_gmap_ids = df[df.duplicated('gmap_id', keep=False)]

# 4. 중복된 gmap_id의 개수
num_duplicated_gmap_ids = duplicated_gmap_ids['gmap_id'].nunique()

# 5. 중복된 행의 수
num_duplicated_rows = duplicated_gmap_ids.shape[0]

# 결과 출력
print(f"전체 행의 수: {total_rows}")
print(f"고유한 gmap_id의 개수: {unique_gmap_ids}")
print(f"고유한 중복된 gmap_id의 개수: {num_duplicated_gmap_ids}")
print(f"중복된 행의 수: {num_duplicated_rows}")

In [0]:
# 중복된 gmap_id 찾기
duplicated_gmap_ids = df[df.duplicated('gmap_id', keep=False)]

# 중복된 gmap_id를 가진 행들을 gmap_id 기준으로 정렬
duplicated_gmap_ids_sorted = duplicated_gmap_ids.sort_values(by='gmap_id')

# 중복된 gmap_id 출력
du_df = duplicated_gmap_ids_sorted

du_df

In [0]:
id_tf = du_df['gmap_id'].value_counts() !=2
id_tf.sum()

In [0]:
du_df.head()

In [0]:
# 중복된 행 제거
df_unique = df.drop_duplicates(subset='gmap_id')

df_unique.shape

In [0]:
df_unique['gmap_id'].nunique()

In [0]:
df = df_unique

## 평균 평점(avg_rating)

### 전체

In [0]:
# avg_rating의 도수분포표 생성
bins = [i * 0.5 for i in range(2, 11)]  # 1.0에서 5.0까지 0.5 단위로 구간 설정
counts, bin_edges = np.histogram(df['avg_rating'], bins=bins)

# 도수분포표 생성
freq_table = pd.DataFrame({
    'Bin Range': [f'{bin_edges[i]} - {bin_edges[i+1]}' for i in range(len(bin_edges)-1)],
    'Frequency': counts
})

# 전체에 대한 비율 계산
total_count = counts.sum()
freq_table['Percentage'] = np.round((freq_table['Frequency'] / total_count) * 100, 2)

freq_table

In [0]:
freq_table['Frequency'].sum()

In [0]:
len(df['avg_rating'])

In [0]:
df['avg_rating'].max(), df['avg_rating'].min()

In [0]:
# avg_rating의 도수분포표 생성
bins = [i * 0.5 for i in range(2, 11)]  # 1.0에서 5.0까지 0.5 단위로 구간 설정
counts, bin_edges = np.histogram(df['avg_rating'], bins=bins)

# 도수분포표 생성
freq_table = pd.DataFrame({
    'Bin Range': [f'{bin_edges[i]} - {bin_edges[i+1]}' for i in range(len(bin_edges)-1)],
    'Frequency': counts
})

# 전체에 대한 비율 계산 및 소수점 둘째 자리까지 반올림
total_count = counts.sum()
freq_table['Percentage'] = np.round((freq_table['Frequency'] / total_count) * 100, 2)

# 히스토그램과 KDE 시각화
plt.figure(figsize=(10, 6))
ax = sns.histplot(df['avg_rating'], bins=bins, kde=True)

# 각 구간에 대한 빈도수와 비율 추가
for i in range(len(bins) - 1):
    plt.text(bin_edges[i] + 0.25, counts[i] + 0.5, f'{counts[i]}\n({freq_table["Percentage"][i]}%)',
             ha='center', va='bottom', fontsize=10, color='black')

# y축 범위 조정
y_max = max(counts) * 1.1
plt.ylim(0, y_max)

plt.title('Average Rating Distribution')
plt.xlabel('Average Rating')
plt.ylabel('Frequency')
plt.show()

### Permanently closed를 제외한 평균 평점 분포

In [0]:
len(df[df['state']=='Permanently closed'])

In [0]:
# Permanently closed가 아닌 데이터 필터링
filtered_df = df[df['state'] != 'Permanently closed']

# avg_rating의 도수분포표 생성
bins = [i * 0.5 for i in range(2, 11)]  # 1.0에서 5.0까지 0.5 단위로 구간 설정
counts, bin_edges = np.histogram(filtered_df['avg_rating'], bins=bins)

# 도수분포표 생성
freq_table = pd.DataFrame({
    'Bin Range': [f'{bin_edges[i]} - {bin_edges[i+1]}' for i in range(len(bin_edges)-1)],
    'Frequency': counts
})

# 전체에 대한 비율 계산 및 소수점 둘째 자리까지 반올림
total_count = counts.sum()
freq_table['Percentage'] = np.round((freq_table['Frequency'] / total_count) * 100, 2)

# 히스토그램과 KDE 시각화
plt.figure(figsize=(10, 6))
ax = sns.histplot(filtered_df['avg_rating'], bins=bins, kde=True)

# 각 구간에 대한 빈도수와 비율 추가
for i in range(len(bins) - 1):
    plt.text(bin_edges[i] + 0.25, counts[i] + 0.5, f'{counts[i]}\n({freq_table["Percentage"][i]}%)',
             ha='center', va='bottom', fontsize=10, color='black')

# y축 범위 조정
y_max = max(counts) * 1.1
plt.ylim(0, y_max)

plt.title('Average Rating Distribution (Excluding Permanently Closed)')
plt.xlabel('Average Rating')
plt.ylabel('Frequency')
plt.show()

### Permanently closed의 평균 평점 분포

In [0]:
# Permanently closed 데이터 필터링
closed_df = df[df['state'] == 'Permanently closed']

# avg_rating의 도수분포표 생성
bins = [i * 0.5 for i in range(2, 11)]  # 1.0에서 5.0까지 0.5 단위로 구간 설정
counts, bin_edges = np.histogram(closed_df['avg_rating'], bins=bins)

# 도수분포표 생성
freq_table = pd.DataFrame({
    'Bin Range': [f'{bin_edges[i]} - {bin_edges[i+1]}' for i in range(len(bin_edges)-1)],
    'Frequency': counts
})

# 전체에 대한 비율 계산 및 소수점 둘째 자리까지 반올림
total_count = counts.sum()
freq_table['Percentage'] = np.round((freq_table['Frequency'] / total_count) * 100, 2)

# 히스토그램과 KDE 시각화
plt.figure(figsize=(10, 6))
ax = sns.histplot(closed_df['avg_rating'], bins=bins, kde=True)

# 각 구간에 대한 빈도수와 비율 추가
for i in range(len(bins) - 1):
    plt.text(bin_edges[i] + 0.25, counts[i] + 0.5, f'{counts[i]}\n({freq_table["Percentage"][i]}%)',
             ha='center', va='bottom', fontsize=10, color='black')

# y축 범위 조정
y_max = max(counts) * 1.1
plt.ylim(0, y_max)

plt.title('Average Rating Distribution (Permanently Closed)')
plt.xlabel('Average Rating')
plt.ylabel('Frequency')
plt.show()

### 4.0 미만의 비율 비교

In [0]:
# 전체 데이터프레임 비율 계산
total_df_count = len(df)
df_below_4 = df[df['avg_rating'] < 4.0]
df_below_4_count = len(df_below_4)
df_below_4_percentage = round((df_below_4_count / total_df_count) * 100, 2)

# Permanently closed가 아닌 데이터 비율 계산
total_filtered_df_count = len(filtered_df)
filtered_df_below_4 = filtered_df[filtered_df['avg_rating'] < 4.0]
filtered_df_below_4_count = len(filtered_df_below_4)
filtered_df_below_4_percentage = round((filtered_df_below_4_count / total_filtered_df_count) * 100, 2)

# Permanently closed 데이터 비율 계산
total_closed_df_count = len(closed_df)
closed_df_below_4 = closed_df[closed_df['avg_rating'] < 4.0]
closed_df_below_4_count = len(closed_df_below_4)
closed_df_below_4_percentage = round((closed_df_below_4_count / total_closed_df_count) * 100, 2)

# 결과를 데이터프레임으로 생성
percentage_df = pd.DataFrame({
    'DataFrame': ['df', 'filtered_df', 'closed_df'],
    'Total Count': [total_df_count, total_filtered_df_count, total_closed_df_count],
    'Count < 4.0': [df_below_4_count, filtered_df_below_4_count, closed_df_below_4_count],
    'Percentage < 4.0': [df_below_4_percentage, filtered_df_below_4_percentage, closed_df_below_4_percentage]
})

percentage_df

## 리뷰 수(num_of_reviews)

In [0]:
# 기술 통계량
df['num_of_reviews'].describe()

In [0]:
# 히스토그램 그리기
plt.figure(figsize=(10, 6))
sns.histplot(df['num_of_reviews'], bins=30, kde=True)  # bins: 히스토그램의 막대 개수, kde: 커널 밀도 추정
plt.title('Distribution of num_of_reviews(ALL)')
plt.xlabel('num_of_reviews')
plt.ylabel('Frequency')

plt.show()

In [0]:
# 구간 설정
bins = [0, 20, 40, 60, 80, 100]
labels = ['0-20', '20-40', '40-60', '60-80', '80-100']

# num_of_reviews 컬럼을 구간별로 나누기
df['binned'] = pd.cut(df['num_of_reviews'], bins=bins, labels=labels, right=False)

# 각 구간별 값들의 개수 계산
bin_counts = df['binned'].value_counts().sort_index().reset_index()
bin_counts.columns = ['Range', 'Count']

# 결과 출력
print(bin_counts)

In [0]:
# 100 이하의 값들만 선택
df_under_28 = df[df['num_of_reviews'] <= 100]

# num_of_reviews 컬럼의 분포 시각화 (100 이하 값들만)
plt.figure(figsize=(10, 6))
sns.histplot(df_under_28['num_of_reviews'], bins=10, kde=True, edgecolor='black')
plt.title('Distribution of Number of Reviews (<=100)')
plt.xlabel('Number of Reviews')
plt.ylabel('Frequency')

plt.show()

In [0]:
# 28 이하의 값들만 선택
df_under_28 = df[df['num_of_reviews'] <= 28]

# num_of_reviews 컬럼의 분포 시각화 (28 이하 값들만)
plt.figure(figsize=(10, 6))
sns.histplot(df_under_28['num_of_reviews'], bins=10, kde=True, edgecolor='black')
plt.title('Distribution of Number of Reviews (<=28)')
plt.xlabel('Number of Reviews')
plt.ylabel('Frequency')

plt.show()

In [0]:
len(df[df['num_of_reviews'] <= 10])

In [0]:
len(df[df['num_of_reviews'] <= 10])/len(df)

In [0]:
len(df[df['num_of_reviews']==0])

In [0]:
# 여러 통계량 계산
min_value = round(df['num_of_reviews'].min(), 2)
max_value = round(df['num_of_reviews'].max(), 2)
mode_value = round(df['num_of_reviews'].mode()[0] if not df['num_of_reviews'].mode().empty else None, 2)
mean_value = round(df['num_of_reviews'].mean(), 2)
median_value = round(df['num_of_reviews'].median(), 2)

# 결과를 데이터프레임으로 정리
summary_data = {
    '통계량': ['최소값', '최대값', '최빈값', '평균값', '중앙값'],
    '값': [min_value, max_value, mode_value, mean_value, median_value]
}

summary_df = pd.DataFrame(summary_data)
summary_df.set_index('통계량', inplace=True)

summary_df

In [0]:
len(df[df['num_of_reviews'] == 8])

In [0]:
df[df['num_of_reviews'] == 8]['gmap_id'].nunique()

## 구글 추천(relative_results)

In [0]:
df['relative_results'].fillna('',inplace=True) #NaN 데이터를 빈칸으로 채움

In [0]:
df['results_count'] = df['relative_results'].apply(len)

In [0]:
df['results_count'].value_counts()

## 리뷰 수와 평점 상관 관계

In [0]:
# 상관계수 계산
correlation = df['num_of_reviews'].corr(df['avg_rating'])

# 상관계수 출력
print(f"Correlation between num_of_reviews and avg_rating: {correlation}")

# 산점도 시각화
plt.scatter(df['num_of_reviews'], df['avg_rating'])
plt.xlabel('Number of Reviews')
plt.ylabel('Average Rating')
plt.title('Scatter Plot of Number of Reviews vs. Average Rating')
plt.show()

In [0]:
# 상관계수 계산
correlation = df[['num_of_reviews', 'avg_rating']].corr()

# 히트맵 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap of num_of_reviews and avg_rating')
plt.show()

## Price

**API docs 기준**
- 0 : free
- 1 : inexpensive
- 2 : moderate
- 3 : expensive
- 4 : very expensive

In [0]:
df['price'].value_counts()

In [0]:
df[df['gmap_id']=='0x7954d370921ff6bd:0x3193ba783e26d032']['address'].iloc[0]

In [0]:
df[df['price']=='$$$'].head(3)

In [0]:
won = df[df['price']=='₩'].head(10)['name']

In [0]:
won.to_list()

In [0]:
won.to_csv('won.csv', index = False)

In [0]:
three = df[df['price']=='$$$'].head(10)['name']

In [0]:
three.to_list()

In [0]:
df.head(3)

In [0]:
df[df['name']=='Kirby Co']

In [0]:
df[df['price'].isnull()].tail()

In [0]:
df[df['name']=='2 Cheesy Guys']

In [0]:
df[df['name']=='2 Cheesy Guys']['hours'].iloc[0]

## Categroy

In [0]:
df[df['category']=='Bar']

In [0]:
df['category'][2000][0].split()

In [0]:
value_counts = df['category'].value_counts()
# count가 10 이상인 데이터 필터링
filtered_value_counts = value_counts[value_counts >= 10]

In [0]:
filtered_value_counts.tail()

In [0]:
pd.set_option('display.max_rows', 10)

In [0]:
cate = df['category'].value_counts()

In [0]:
# 각 카테고리의 빈도 계산
category_counts = df['category'].value_counts()

# 빈도가 10 이상인 카테고리 필터링
categories_with_10_or_more = category_counts[category_counts == 1]

# 이 카테고리들의 비율 계산
proportion = len(categories_with_10_or_more) / len(category_counts)

In [0]:
print(proportion)

In [0]:
len(df['category'][3000])

In [0]:
type(df['category'][0])

In [0]:
# 찾고자 하는 리스트 값
target_value = ['Gay bar', 'Restaurant']

# 'category' 컬럼이 target_value와 같은 행을 필터링
filtered_df = df[df['category'].apply(lambda x: x == target_value)]

filtered_df

In [0]:
len(df['category'][7750])

In [0]:
df['cate_count'] = df['category'].apply(lambda x: len(x) if x is not None else 0)

In [0]:
df['cate_count'].value_counts()

In [0]:
eleven = df[df['cate_count']==11]['category']

In [0]:
df[df['cate_count']==11]

In [0]:
for _ in eleven:
  print(_)

In [0]:
import matplotlib.pyplot as plt

# 히스토그램 생성
plt.figure(figsize=(12, 6))
plt.hist(df['cate_count'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of cate_count')
plt.xlabel('cate_count')
plt.ylabel('Frequency')

# x축 눈금 설정
plt.xticks(range(int(df['cate_count'].min()), int(df['cate_count'].max()) + 1))

plt.show()

In [0]:
import matplotlib.pyplot as plt

# cate_count의 값별 빈도수 계산
value_counts = df['cate_count'].value_counts().sort_index()

# 막대 그래프 생성
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(value_counts.index, value_counts.values, edgecolor='k', alpha=0.7)
ax.set_title('Distribution of cate_count')
ax.set_xlabel('cate_count')
ax.set_ylabel('Frequency')

# x축 눈금 설정
ax.set_xticks(range(int(df['cate_count'].min()), int(df['cate_count'].max()) + 1))

# 막대 위에 숫자와 비율 표시
total_counts = value_counts.sum()
for bar in bars:
    height = bar.get_height()
    x = bar.get_x() + bar.get_width() / 2
    # 숫자 표시
    ax.text(x, height, f'{int(height)}', ha='center', va='bottom')
    # 비율 표시
    ax.text(x, height + 0.05 * total_counts, f'{height/total_counts:.1%}', ha='center', va='bottom', fontsize=8, color='blue')

plt.show()

In [0]:
one_cate = df[df['cate_count']==1]

In [0]:
one_cate['split'] = one_cate['category'].apply(lambda x: x[0].split())

In [0]:
one_cate = one_cate.reset_index(drop=True)

In [0]:
# 확인할 문자열
target = 'shop'

# for문을 사용하여 각 리스트를 검사
for i in range(50):
    if target in one_cate['split'][i]:
        print(f"{i} -> 있음")
    else:
        print(f"{i} -> -")